In [177]:
import requests
import time
import json
from pprint import pprint
import os, sys
import pydicom

sys.path.append("../../../") # Path containing IMPIT library
from impit.dicom.communication import DicomConnector


base_url = 'http://localhost:8000'
api_key = 'f0237de6-d92c-4eaf-9b41-b9bc61b9d9c1'
algorithm_name = 'Cardiac Segmentation'
api_dicom_location = '{0}/api/dicomlocation'.format(base_url)
api_dataset = '{0}/api/dataset'.format(base_url)
api_dataset_ready = '{0}/api/dataset/ready'.format(base_url)
api_data_object = '{0}/api/dataobject'.format(base_url)
api_trigger = '{0}/api/trigger'.format(base_url)
api_algorithm = '{0}/api/algorithm'.format(base_url)

In [178]:
# Get the algorithm and the default settings
algorithm = None
r = requests.get(api_algorithm, headers={'API_KEY': api_key})
if r.status_code == 200:
    for a in r.json():
        pprint(a)
        if a['name'] == algorithm_name:
            algorithm = a
    print("")
    print("Look's Good!")
else:
    print("Oops, something went wrong. Ensure the service is running at the base_url configured and that the API Key has been generated and set in api_key.")

{'default_settings': {'IARSettings': {'minBestAtlases': 4,
                                      'outlierFactor': 1.5,
                                      'outlierMethod': 'IQR',
                                      'referenceStructure': 'COR',
                                      'smoothDistanceMaps': True,
                                      'smoothSigma': 1,
                                      'zScoreStatistic': 'MAD'},
                      'atlasSettings': {'atlasIdList': ['04D1',
                                                        '1FA5',
                                                        '7AAC',
                                                        '8505',
                                                        '5FBF'],
                                        'atlasPath': '../../TempCardiacData',
                                        'atlasStructures': ['COR', 'LAD']},
                      'deformableSettings': {'iterationStaging': [75,
                    

In [179]:
# Create a new Dataset
dataset = None
r = requests.post(api_dataset, headers={'API_KEY': api_key}, data={})
if r.status_code >= 200:
        dataset = r.json()
        
pprint(dataset)

{'from_dicom_location': None,
 'from_dicom_location_id': None,
 'id': 41,
 'input_data_objects': [],
 'output_data_objects': [],
 'owner_key': 'f0237de6-d92c-4eaf-9b41-b9bc61b9d9c1',
 'timeout': '2019-08-20T14:11:45.248',
 'timestamp': '2019-08-19T04:11:45.250',
 'to_dicom_location': None,
 'to_dicom_location_id': None}


In [180]:
# Add a Dicom file to the dataset
ID = 'FF22'
path_to_nii = f'../../TempCardiacData/Case_{ID}/Images/Case_{ID}.nii.gz'
data_object = None

# Get the Series UID of this Data Object
with open(path_to_nii,'rb') as file:

    data = {'dataset': dataset['id'],
            'type': 'FILE',
            'file_name': 'case_03ad.nii.gz'}
    
    r = requests.post(api_data_object, headers={'API_KEY': api_key}, data=data, files={'file_data':file})
    if r.status_code >= 200:
            data_object = r.json()
        
pprint(data_object)

{'children': [],
 'dataset_id': 41,
 'id': 65,
 'is_fetched': True,
 'is_input': True,
 'is_sent': False,
 'meta_data': None,
 'parent': None,
 'parent_id': None,
 'path': '/tmp/tmpbtd1icz5/case_03ad.nii.gz',
 'series_instance_uid': None,
 'status': None,
 'timestamp': '2019-08-19T04:11:45.920',
 'type': 'FILE'}


In [181]:
r = requests.get('{0}/{1}'.format(api_data_object, data_object['id']), headers={'API_KEY': api_key})
if r.status_code == 200:
    data_object = r.json()
    
if data_object['is_fetched']:
    print('The server has the data and is ready!')
else:
    print('The server is still receiving the data or something has gone wrong.')

The server has the data and is ready!


In [182]:
r = requests.get('{0}/{1}'.format(api_dataset_ready, dataset['id']), headers={'API_KEY': api_key})
if r.status_code == 200:
    if r.json()['ready']:
        print("The Dataset is ready, let's run the algorithm!")
    else:
        print("Nope, the dataset isn't ready yet")

The Dataset is ready, let's run the algorithm!


In [ ]:
# Trigger the algorithm with our dataset containing the data object
data={'dataset': dataset['id'],
     'algorithm': 'Cardiac Segmentation'}
r = requests.post(api_trigger, headers={'API_KEY': api_key}, data=data)

if r.status_code == 200:
    # Poll the URL given to determine the progress of the task
    poll_url = '{0}{1}'.format(base_url, r.json()['poll'])
    
    while(1):
        r = requests.get(poll_url, headers={'API_KEY': api_key})
        status = r.json()
        print(status)

        if status['state'] == 'SUCCESS' or status['state'] == 'FAILURE':
            break

        time.sleep(2)
else:
    print(r.json())
    
print('Algorithm Processing Complete')

{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'RUNNING', 'current': 0, 'total': 1, 'status': 'Running Algorithm: Cardiac Segmentation'}
{'state': 'RUNNING', 'current': 0, 'total': 1, 'status': 'Running Algorithm: Cardiac Segmentation'}
{'state': 'RUNNING', 'current': 0, 'total': 1, 'status': 'Running Algorithm: Cardiac Segmentation'}
{'state': 'RUNNING', 'current': 0, 'total': 1, 'status': 'Running Algorithm: Cardiac Segmentation'}
{'state': 'RUNNING', 'current': 0, 'total': 1, 'status': 'Running Algorithm: Cardiac Segmentation'}
{'state': 'RUNNING', 'current': 0, 'total': 1, 'status': 'Running Algorithm: Cardiac Segmentation'}
{'state': 'RUNNING', 'current': 0, 'total': 1, 'status': 'Running Algorithm: Cardiac Segmentation'}
{'state': 'RUNNING', 'current': 0, 'total': 1, 'status': 'Running Algorithm: Cardiac Segmentation'}
{'state': 'RUNNING', 'current': 0, 'total': 1, 'status': 'Running Algorithm: Cardiac Segmentation'}
{'state': 'RUNNING', 'current

In [176]:
# Fetch the latest dataset to see the output objects and download the Nifti file!
r = requests.get('{0}/{1}'.format(api_dataset, dataset['id']), headers={'API_KEY': api_key})
if r.status_code == 200:
    dataset = r.json()
    pprint(dataset)

    for d in dataset['output_data_objects']:
        if d['path'].endswith('nii.gz'):
            #print(d)
            r = requests.get('http://localhost:8000/api/dataobject/download/{0}'.format(d['id']), headers={'API_KEY': api_key})
            filename = r.headers['Content-Disposition'].split('filename=')[1]
            print('Downloading to: {0}'.format(filename))
            open(filename, 'wb').write(r.content)

{'from_dicom_location': None,
 'from_dicom_location_id': None,
 'id': 40,
 'input_data_objects': [{'children': [{'children': [],
                                       'dataset_id': 40,
                                       'id': 63,
                                       'is_fetched': False,
                                       'is_input': False,
                                       'is_sent': False,
                                       'meta_data': None,
                                       'parent_id': 62,
                                       'path': '/tmp/tmp9z5box6s/Auto_COR.nii.gz',
                                       'series_instance_uid': None,
                                       'status': None,
                                       'timestamp': '2019-08-19T04:10:33.124',
                                       'type': 'FILE'},
                                      {'children': [],
                                       'dataset_id': 40,
                       

In [67]:
r = requests.get('{0}/{1}'.format(api_dataset_ready, dataset['id']), headers={'API_KEY': api_key})
r.json()

{'ready': True}